In [ ]:
from ipywidgets import AppLayout, Button, Layout, Image

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from ipywidgets import HBox, VBox
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout
import ipywidgets as widgets
import scipy.io
import numpy as np
import tkinter
from sys import executable, argv
from subprocess import check_output
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
from ipywidgets import HTML
import os
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import seaborn as sns
import markdown 
from matplotlib.widgets import Slider, Button, RadioButtons


In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

final_graph_inp_arr = np.load('final_graph_inp_arr.npy')
final_graph_inp_arr.shape

# restore np.load for future normal usage
np.load = np_load_old
final_graph_inp_arr.shape

all_arr = []
for x in final_graph_inp_arr:
    if len(x) > 1:
        all_arr.append(np.array(x))

all_arr = np.array(all_arr[1:])
all_arr.shape

unique_st = np.unique(all_arr[:,2:3])[:-1]

sets = []

for st in unique_st: 
    sets.append([])
    #print(st)
    for each_record in all_arr:
        try:
            if st == each_record[2]:        
                sets[-1].append(each_record)
        except:
            continue
            
sets_arr = np.array(sets)
sets_arr.shape

city_names = open('city_coordinates.txt',encoding="utf8").readlines()
city_coords = []
for lines in city_names:
    city_coords.append([lines.replace('\n','').split(',')[1],float(lines.replace('\n','').split(',')[2]),float(lines.replace('\n','').split(',')[3])])


In [ ]:
html = markdown.markdown("""<html>
<head>
<link href='https://fonts.googleapis.com/css?family=Roboto' rel='stylesheet'>
<style>
body {
    font-family: 'Roboto';font-size: 22px;
}
</style>
</head>
<body>


  <h1 style="text-align: center;"><span style="font-size: 
      26px;">CMP684 Project</span></h1>
  <h1 style="text-align: center;"><span style="font-size: 26px;"><strong>V<sub>s30</sub> Prediction from Earthquake Accelerograms</strong></span></h1>
  <p style="text-align: justify;"><span style="font-size: 19px;"><strong><span class="fontstyle0" style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial;">Earthquakes are recorded with seismograph devices all over the world especially at countries with high seismicity (eg. Turkey, Japan). Records of these devices, namely accelerograms (t</span></strong><strong><span style="float: none; color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial; display: inline !important;">he recording of the acceleration of the ground during an earthquake</span></strong><strong><span class="fontstyle0" style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial;">), vaguely reflects some characteristics of seismograph site.</span></strong><strong><span style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial; float: none; display: inline !important;">&nbsp;In this study, a parameter of site, average shear wave velocity of top 30 meters of soil (V<sub>s30</sub>) predicted using accelerograms recorded at the same site with the help of a neural network.</span></strong></span></p>
  <p style="text-align: center;"><span style="font-size: 19px;"><strong><span style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial; float: none; display: inline !important;">
          <img src="..." style="width: 637px;">
        </span></strong></span></p>
  <p style="text-align: center;"><span style="font-size: 16px;"><strong><span style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: start; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; background-color: rgb(255, 255, 255); text-decoration-style: initial; text-decoration-color: initial; float: none; display: inline !important;"><em>AFAD strong ground motion seismograph network [1].</em></span></strong></span></p>
  <p style="text-align: justify;"><span style="font-size: 19px; font-family: Tahoma, Geneva, sans-serif;">In Turkey there are 679 seismograph devices actively working in AFAD's (Disaster &amp; Emergency Management Authority) strong ground motion seismograph network.</span><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">&nbsp;For 68 of those 679 seismographs the geological borehole log reports are available online [2] and for 258 seismographs the V<sub>s30</sub> value of site is known. V<sub>s30</sub> is accepted as a indicator of the hardness of soil of a site. If site has rock-type properties, V<sub>s30</sub> is high and if site has soil-type properties V<sub>s30</sub> is low. An example soil classification according to V<sub>s30&nbsp;</sub>values is shown below:</span></p>
  <p style="text-align: justify;">
    <br>
  </p>



</body>
</html>""")
HTML(html) 


In [ ]:
html = markdown.markdown("""<html>
<head>
<link href='https://fonts.googleapis.com/css?family=Roboto' rel='stylesheet'>
<style>
body {
    font-family: 'Roboto';font-size: 22px;
}
</style>
</head>
<body>

<p style="text-align: center;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">
    <img src="..." style="width: 371px;">
  </span></p>
<p style="text-align: center;"><em>Soil classification table according to V<sub>s30</sub> values [3]</em></p>
<p style="text-align: center;">
  <br>
</p>
<p style="text-align: center;">
  <img src="..." style="width: 436px;">
</p>
<p style="text-align: center;"><em>Shear wave propagation in soil medium [4].</em></p>
<p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;"><span style="color: rgb(0, 0, 0); font-family: Tahoma, Geneva, sans-serif; font-size: 19px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; font-weight: 400; letter-spacing: normal; orphans: 2; text-align: justify; text-indent: 0px; text-transform: none; white-space: normal; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; text-decoration-style: initial; text-decoration-color: initial; display: inline !important; float: none;">In this study,</span> earthquakes recorded by seismographs with known V<sub>s30</sub> values are used. By the date 26.11.2019 (cut-off date for data gathering) there were around 20.000 earthquake records (Magnitude ≥ 3) caught by these 258 seismographs. Each of these records have three accelerograms in three perpendicular directions (north-south (<em>ns</em>), east-west (<em>ew</em>), up-down(<em>ud</em>)). A sample record shown below.</span></p>
<p style="text-align: center;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">
    <img src="..." style="width: 639px;">
  </span></p>
<p style="text-align: center;"><em>North-South, East-West and Up-Down records of earthquake recorded at station no 1009 at 2008.</em></p>
<p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">These records are not used directly in the training of neural network model. In order to standardize the input and to reflect the frequency dependent behavior better, accelerogram records converted to spectra using Niggam-Jennings methodology [5]. Spectra calculated for sample accelerograms are given below.</span></p>
<p style="text-align: center;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">
    <br>
    <img src="..." style="width: 675px;">
  </span></p>

</body>
</html>""")
HTML(html) 

In [ ]:
html = markdown.markdown("""<html>
<head>
<link href='https://fonts.googleapis.com/css?family=Roboto' rel='stylesheet'>
<style>
body {
    font-family: 'Roboto';font-size: 22px;
}
</style>
</head>
<body>

  <p style="text-align: center;"><em>Spectra calculated for North-South, East-West and Up-Down records of earthquake recorded at station no 1009 at 2008.</em></p>
  <p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;"><span class="fontstyle0">Using the spectra derived from over 20.000 earthquake records a neural network model formed and trained in a way to <strong><u>extract&nbsp;</u></strong></span><strong><u><span class="fontstyle2">V</span><span class="fontstyle3"><sub>s</sub></span><span class="fontstyle4"><sub>30</sub>&nbsp;</span></u></strong><span class="fontstyle0"><u><strong>value from the relation between horizontal and vertical components of earthquake recordings.</strong></u> Thus North-South_Up-Down (<em>ns-ud</em> ) and East-West_Up-Down (<em>ew-ud</em> ) spectrum couples used as input and&nbsp;</span><span class="fontstyle2">V</span><span class="fontstyle3"><sub>s</sub></span><span class="fontstyle4"><sub>30</sub>&nbsp;</span><span class="fontstyle0">value used as output.</span>&nbsp;</span></p>
  <p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;"><u><strong>Neural Network Model</strong></u></span></p>
  <p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">Schematic of neural network model is given below:</span></p>
  <p style="text-align: center;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">
      <img src="...">&nbsp; 
    </span><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 16px;"><em>&nbsp;Neural network model used in this study.</em></span></p>
  <p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;"><u><strong>Predictions:</strong></u></span></p>
  <p style="text-align: justify;"><span style="font-family: Tahoma, Geneva, sans-serif; font-size: 19px;">At the below graph prediction results of the neural network model can be reviewed. At each graph blue circle shows the station and all other circles represents the earthquakes recorded by that station. Colors of earthquake circles changes via error in prediction. Size of each circle represents the magnitude of the corresponding earthquake. And finally real and predicted V<sub>s30</sub> values can be seen with mouse hover circle (with related radiobutton selected).</span></p>
  <p style="text-align: center;">
    <br>
  </p>
  <p style="text-align: justify;">
    <br>
  </p>


</body>
</html>""")
HTML(html) 

In [ ]:
#grid = GridspecLayout(1, 5, height='50px')
#grid[0, 0] = create_expanded_button('Main', 'info')
#grid[0, 1] = create_expanded_button('Input', 'info')
#grid[0, 2] = create_expanded_button('Analysis', 'info')
#grid[0, 3] = create_expanded_button('Plot', 'info')
#grid[0, 4] = create_expanded_button('Other', 'info')
#display(grid)

In [ ]:

#x = np.random.rand(15)
#y = np.random.rand(15)
#names = np.array(list("ABCDEFGHIJKLMNO"))
#c = np.random.randint(1,5,size=15)

#norm = plt.Normalize(1,4)
#cmap = plt.cm.RdYlGn

#fig,ax = plt.subplots()
#sc = plt.scatter(x,y,c=c, s=100, cmap=cmap, norm=norm)





In [ ]:
%matplotlib notebook


#fig = plt.figure()
#ax = fig.add_subplot(111)
fig,ax = plt.subplots(num='Vs30 Predictions')




def f_slider(station): 
    #try:
        global points, points2
        #1. çalışan
        
        stt = cfeature.NaturalEarthFeature(category='cultural', 
            name='admin_0_boundary_lines_land',
            scale='10m',facecolor='none')
        stt_prv = cfeature.NaturalEarthFeature(category='cultural', 
            name='admin_1_states_provinces_lines',
            scale='10m',facecolor='none')


        st_no = station
        my_set = np.array(sets_arr[st_no])
        my_set_nsud = []
        my_set_ewud = []
        for i in range(len(sets_arr[st_no])):
            if i % 2 == 0:
                my_set_nsud.append(sets_arr[st_no][i])
            else:
                my_set_ewud.append(sets_arr[st_no][i])

        my_set_nsud = np.array(my_set_nsud)
        my_set_nsud.shape        

        my_set_ewud = np.array(my_set_ewud)
        my_set_ewud.shape

        #print(my_set.shape[0])
        #print(my_set[:,13:14])
        my_set.shape
        st_n = float(my_set[:,6:7][0])
        st_e = float(my_set[:,7:8][0])
        eq_n = np.array(my_set[:,8:9],dtype=float).reshape(-1,)
        eq_e = np.array(my_set[:,9:10],dtype=float).reshape(-1,)
        res = 5

        #print(BBox)        
        #cmap = sns.cubehelix_palette(as_cmap=True)
        # Generate a custom diverging colormap
        #cmap = sns.diverging_palette(220, 10, as_cmap=True)
        #cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
        #plt.figure(figsize=(12, 6))
        def update_annot(ind):

            pos = points.get_offsets()[ind["ind"][0]]
            annot.xy = pos
            if my_rdbtn.value == 1:
                text = "{}".format(np.array(my_set_nsud[:,3:4],dtype=float)[int(list(map(str,ind["ind"]))[0])][0])
            elif my_rdbtn.value == 2:
                text = "{}".format(np.array(my_set[:,0:1])[int(list(map(str,ind["ind"]))[0])][0])
            elif my_rdbtn.value == 3:
                text = "R: {} NS: {} EW: {}".format(round(np.array(my_set[:,13:14],dtype=float)[int(list(map(str,ind["ind"]))[0])][0],0),
                                            round(np.array(my_set_nsud[:,14:15],dtype=float)[int(list(map(str,ind["ind"]))[0])][0],0),
                                           round(np.array(my_set_ewud[:,14:15],dtype=float)[int(list(map(str,ind["ind"]))[0])][0],0))
                
            annot.set_text(text)
            #annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
            annot.get_bbox_patch().set_alpha(0.4)


        def hover(event):
            vis = annot.get_visible()
            if event.inaxes == ax:
                cont, ind = points.contains(event)
                if cont:
                    update_annot(ind)
                    annot.set_visible(True)
                    fig.canvas.draw_idle()
                else:
                    if vis:
                        annot.set_visible(False)
                        fig.canvas.draw_idle()

        #fig, ax = plt.subplots()
        fig.clf()
        prr = ccrs.PlateCarree(globe=None)

        ax = plt.axes(projection=prr)

        my_n = np.array(my_set[:,8:9],dtype=float)
        my_e = np.array(my_set[:,9:10],dtype=float)
        my_n_nsud = np.array(my_set_nsud[:,8:9],dtype=float)
        my_e_nsud = np.array(my_set_nsud[:,9:10],dtype=float)
        my_n_ewud = np.array(my_set_ewud[:,8:9],dtype=float)
        my_e_ewud = np.array(my_set_ewud[:,9:10],dtype=float)
        my_mag_nsud = np.array(my_set_nsud[:,3:4],dtype=float)**3.5
        my_mag_ewud = (np.array(my_set_nsud[:,3:4],dtype=float)**3.5) / 3  # magnitude iki yönde de aynı olduğu için ikisinde de nsud
        my_c_nsud = np.abs(np.array(my_set_nsud[:,13:14],dtype=float) - np.array(my_set_nsud[:,14:15],dtype=float))/np.array(my_set_nsud[:,13:14],dtype=float) 
        my_c_ewud = np.abs(np.array(my_set_ewud[:,13:14],dtype=float) - np.array(my_set_ewud[:,14:15],dtype=float))/np.array(my_set_ewud[:,13:14],dtype=float) 
        my_c_nsud = np.clip(my_c_nsud, 0, 0.5) # dikkat max error u 0.5 de fixliyor. Colorbar için gerekti.
        my_c_ewud = np.clip(my_c_ewud, 0, 0.5) 
        BBox = [np.min(my_e_nsud)-1, np.max(my_e_nsud)+1, np.min(my_n_nsud)-1, np.max(my_n_nsud)+1] #ewud veya nsud farketmediği (aynı koordinatlar) için nsud yaptım
        my_c = np.abs(np.array(my_set[:,13:14],dtype=float) - np.array(my_set[:,14:15],dtype=float))/np.array(my_set[:,13:14],dtype=float)
        ax.set_extent(BBox)
        my_c = np.clip(my_c, 0, 0.5) 
        my_c = np.append(my_c, [0,0.5])
        my_c_nsud = np.append(my_c_nsud, [0,0.5])
        my_c_ewud = np.append(my_c_ewud, [0,0.5])
        my_n_nsud = np.append(my_n_nsud, [1,2]) # dummy points for colorbar scale correction
        my_n_ewud = np.append(my_n_ewud, [1,2])
        my_e_nsud = np.append(my_e_nsud, [1,2]) # dummy points for colorbar scale correction
        my_e_ewud = np.append(my_e_ewud, [1,2])
        my_mag_nsud = np.append(my_mag_nsud, [1,2])
        my_mag_ewud = np.append(my_mag_ewud, [1,2])


        cmap = sns.diverging_palette(160, 10, s=80, l=50, n=90, as_cmap=True)
        points = plt.scatter(my_e_nsud, my_n_nsud, zorder=1, alpha= 0.9,c = my_c_nsud, s=my_mag_nsud, cmap=cmap)
        points2 = plt.scatter(my_e_ewud, my_n_ewud, zorder=1, alpha= 0.9,c = my_c_ewud, s=my_mag_ewud, cmap=cmap)
        ax.scatter(st_e, st_n, zorder=1, alpha= 0.8, c='b', s=100)
        ax.text(st_e, st_n, str(my_set[0][2]), size=9, ha='left', va='top',position=(st_e*1.004, st_n*1.004))
        #ax.scatter(st_e, st_n, zorder=1, alpha= 0.8, c=0, s=100)
        #ax.scatter(st_e, st_n, zorder=1, alpha= 0.8, c=0.5, s=100)
        ax.set_title("Station no: " + str(my_set[0][2]) + " # of eqs: " + str(my_set.shape[0]))       





        fig.canvas.mpl_connect("motion_notify_event", hover)

        fig.colorbar(points, label='Error')        
        
        annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
            bbox=dict(boxstyle="round", fc="w"),
            arrowprops=dict(arrowstyle="->"))
        annot.set_visible(False)
        ax.gridlines()
        ax.coastlines('10m')
        ax.add_feature(stt,     linewidth=0.5, edgecolor='black', zorder=0)
        ax.add_feature(stt_prv, linewidth=0.2, edgecolor='black', zorder=5)
        for i in range(len(city_coords)):
            if city_coords[i][2] > BBox[0] and city_coords[i][2] < BBox[1] and city_coords[i][1] > BBox[2] and city_coords[i][1] < BBox[3]: # kutu dışına çıkan isimleri engellemek için
                ax.text(city_coords[i][2], city_coords[i][1], city_coords[i][0], size=7, ha='center', va='center', color='grey')
            #ax.annotate(city_coords[i][0], (city_coords[i][2], city_coords[i][1]))
        plt.show()
        #fig.draw()

    #except Exception as e:
        #print(e)
        #pass

f_slider(110)
slider=widgets.IntSlider(min=0, max=223, step=1, value=110)    
#slider = widgets.IntSlider()
#display(slider)


interactive_plot = interactive(f_slider, station=slider)

bck_button=widgets.Button(description='',button_style='',icon='chevron-left',layout=Layout(width='40px'))
fwd_button=widgets.Button(description='',button_style='',icon='chevron-right',layout=Layout(width='40px'))

AppL1 = AppLayout(header=None,
          left_sidebar=bck_button,
          center=slider,
          right_sidebar=fwd_button,
          footer=None,
          pane_widths=['40px', '300px', '40px'],
          pane_heights=[1, 1, 1])


my_rdbtn = widgets.RadioButtons(
    options=[('Magnitude', 1),('Eq_id', 2), ('Vs30', 3)],
#    value='pineapple', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Annotations:',
    disabled=False
)
#display(my_rdbtn)
grid0 = GridspecLayout(1, 5, height='100px',width='100%',justify_items='center',align_items = 'center')
grid0[0, 2:4] = AppL1

grid0[0, 0:2] = my_rdbtn
#grid0[0, 1] = slider
#grid0[0, 2] = fwd_button
display(grid0)
#display(AppL1)
    
def bck_click(b):
    slider.value=slider.value-1
    
def fwd_click(b):
    slider.value=slider.value+1
    
    
bck_button.on_click(bck_click)
fwd_button.on_click(fwd_click)

#output = interactive_plot.children[-1]

#display(interactive_plot)
    
    

#click(None) # boş çalıştırmak için




In [ ]:
html = markdown.markdown("""<html>
<head>
<link href='https://fonts.googleapis.com/css?family=Roboto' rel='stylesheet'>
<style>
body {
    font-family: 'Roboto';font-size: 22px;
}
</style>
</head>
<body>

  <p style="text-align: justify;"><span style="font-size: 19px; font-family: Tahoma, Geneva, sans-serif;">This study done by Ali AKGÖZ as a term project for 
      <a href="https://web.cs.hacettepe.edu.tr/~onderefe/bil684/" rel="noopener noreferrer" target="_blank">CMP684 - Neural Networks Course</a> (Hacettepe University - Computer Engineering) under supervision of the instructor of the course 
      <a href="https://web.cs.hacettepe.edu.tr/~onderefe/" rel="noopener noreferrer" target="_blank">Prof. Dr. M.Önder EFE</a>.
    </span></p>
  <p>
    <br>
  </p>
  <p>
    <br>
  </p>
  <p style="text-align: justify;"><strong>References:</strong></p>
  <p style="text-align: justify;">[1] 
    <a href="https://deprem.afad.gov.tr/istasyonlar">https://deprem.afad.gov.tr/istasyonlar</a> &nbsp;
  </p>
  <p style="text-align: justify;">[2] 
    <a href="http://kyhdata.deprem.gov.tr/">http://kyhdata.deprem.gov.tr/</a>&nbsp; (currently unavailable due to maintenance. 17.03.2020)&nbsp;
  </p>
  <p style="text-align: justify;">[3] Building Seismic Safety Council, NEHRP Recommended Provisions for Seismic Regulations for New Buildings and Other Structures (Fema 450)," Part 1, no. Fema 450, p. 338, 2003.</p>
  <p style="text-align: justify;">[4] 
    <a href="https://web.ics.purdue.edu/~braile/edumod/waves/Swave.gif">https://web.ics.purdue.edu/~braile/edumod/waves/Swave.gif</a> &nbsp;
  </p>
  <p style="text-align: justify;">[5] N. Nigam, P. Jennings, and C. I. of Technology Earthquake Engineering Research Laboratory, Digital calculation of response spectra from strong-motion earthquake records. California Institute of Technology, Earthquake Engineering Research Laboratory, 1968</p>

</body>
</html>""")
HTML(html) 